# import lib

In [1]:
import os
import math
import datetime
import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import TFBertForSequenceClassification

from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

from sklearn.metrics import confusion_matrix, classification_report


from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib import rc


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


In [3]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [77]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 1
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
MAX_LEN = 256

# create a synthetic data set

In [180]:
import pandas as pd
import random

# Define the intents and some example phrases for each intent
intents = {
    "add_task": [
        "Add a new task to my list",
        "Please add a reminder to buy groceries",
        "Create a task for calling John",
        "Add task to clean the house",
        "Set a new task for paying bills",
        "I need to add a task for picking up laundry",
        "Can you add a reminder for meeting at 5 PM",
        "Include a task for car maintenance",
        "Add a task to water the plants",
        "Please add a task for project deadline",
    ],
    "list_tasks": [
        "Show all my tasks",
        "List all tasks for today",
        "What tasks do I have?",
        "Give me the task list",
        "I want to see my tasks",
        "List all tasks on my agenda",
        "Display pending tasks",
        "Show tasks due this week",
        "Can I see the tasks I have?",
        "List all my current tasks",
    ],
    "complete_task": [
        "Mark task 3 as complete",
        "Finish task 2",
        "Complete the task about groceries",
        "I have completed task 4",
        "Mark task number 5 done",
        "Close task related to laundry",
        "Check off task to call John",
        "Task 6 is finished",
        "Task 7 is completed",
        "Complete the meeting task",
    ],
    "delete_task": [
        "Delete task 1 from my list",
        "Remove the grocery task",
        "Erase task number 2",
        "Can you delete the cleaning task?",
        "Remove task about project deadline",
        "Delete task related to meeting",
        "Clear task to water plants",
        "Erase task 4",
        "Delete the call John task",
        "Remove task number 3",
    ]
}

# Generate a larger dataset by repeating examples to reach 40 examples per intent
data = []
for label, examples in intents.items():
    for _ in range(4):  # Repeat each set of examples 4 times to get 40 examples per intent
        for example in examples:
            data.append({"Text": example, "Label": label})

# Shuffle and expand data by repeating the dataset 10 times to reach 1600 total examples
random.shuffle(data)
data = data * 10  # Multiply the 160 examples by 10 to get 1600 examples

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("intent_classification_dataset.csv", index=False)

print("Dataset created and saved as intent_classification_dataset.csv")


Dataset created and saved as intent_classification_dataset.csv


# read data

In [117]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/intent_classification_dataset.csv')

# Split data into train (70%) and temp (30%) for validation and test
train, temp_df = train_test_split(data, test_size=0.3, random_state=42)

# Split temp into validation (33% of 30% = 10% of total) and test (67% of 30% = 20% of total)
valid, test = train_test_split(temp_df, test_size=0.67, random_state=42)

# Check the sizes
print("Training Set Size:", len(train))
print("Validation Set Size:", len(valid))
print("Test Set Size:", len(test))

Training Set Size: 1120
Validation Set Size: 158
Test Set Size: 322


In [118]:
train.head()


,Text,Label
1247,Finish task 2,complete_task
1326,Create a task for calling John,add_task
679,Remove task about project deadline,delete_task
845,Erase task number 2,delete_task
926,Give me the task list,list_tasks


In [119]:
valid.head()

,Text,Label
382,I have completed task 4,complete_task
271,Add a new task to my list,add_task
1236,Please add a task for project deadline,add_task
48,Complete the meeting task,complete_task
720,Clear task to water plants,delete_task


In [169]:
train.shape

(1120, 1)

In [121]:
train_df=train.iloc[:,1:]
train

,Text,Label
1247,Finish task 2,complete_task
1326,Create a task for calling John,add_task
679,Remove task about project deadline,delete_task
845,Erase task number 2,delete_task
926,Give me the task list,list_tasks
...,...,...
1130,Show all my tasks,list_tasks
1294,List all tasks for today,list_tasks
860,Complete the meeting task,complete_task
1459,Check off task to call John,complete_task


In [122]:
#test=test.iloc[:,1:]
test
#valid=valid.iloc[:,1:]

,Text,Label
817,Remove the grocery task,delete_task
1514,List all my current tasks,list_tasks
722,I have completed task 4,complete_task
1436,I want to see my tasks,list_tasks
857,Create a task for calling John,add_task
...,...,...
575,Delete task 1 from my list,delete_task
297,What tasks do I have?,list_tasks
968,Task 7 is completed,complete_task
1087,Finish task 2,complete_task


In [123]:
train_y=train['Label'].astype('category').cat.codes
valid_y=valid['Label'].astype('category').cat.codes
test_y=test['Label'].astype('category').cat.codes

train_y.head()

,0
1247,1
1326,0
679,2
845,2
926,3


In [124]:
valid_y.head()

,0
382,1
271,0
1236,0
48,1
720,2


In [125]:
train.drop(columns=['Label'],inplace=True)
valid.drop(columns=['Label'],inplace=True)
test.drop(columns=['Label'],inplace=True)
train

,Text
1247,Finish task 2
1326,Create a task for calling John
679,Remove task about project deadline
845,Erase task number 2
926,Give me the task list
...,...
1130,Show all my tasks
1294,List all tasks for today
860,Complete the meeting task
1459,Check off task to call John


In [126]:
train_y=train_y.astype('int64')
test_y=test_y.astype('int64')
valid_y=valid_y.astype('int64')

# get encoding values for text

In [127]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def regular_encode(texts, tokenizer, maxlen=256):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_masks=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen    )

    return np.array(enc_di['input_ids' ])

In [128]:
valid.head()

,Text
382,I have completed task 4
271,Add a new task to my list
1236,Please add a task for project deadline
48,Complete the meeting task
720,Clear task to water plants


In [129]:
x_train = regular_encode(train.Text	.values, tokenizer, maxlen=MAX_LEN)
x_valid = regular_encode(valid.Text	.values, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test.Text.values, tokenizer, maxlen=MAX_LEN)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_atte

# prepare train,val,test data for model use

In [130]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, train_y.values))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO) )

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, valid_y.values))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO) )

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_test,test_y.values))
    .batch(BATCH_SIZE) )

# model building

In [ ]:
### NOOTTTT USE thids
def build_model(transformer, max_len=100):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_token)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=7, activation="softmax")(logits)

    model = Model(inputs=input_word_ids, outputs=logits)
    model.compile(optimizer=keras.optimizers.Adam(1e-5),loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")] )

    return model

In [85]:
##### NOTTT use this model
def build_model(transformer, max_len=256):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")

    # Use the functional API directly within the Keras Model
    transformer_output = Lambda(lambda x: transformer(x)[0])(input_word_ids)

    # Get the CLS token (sequence_output)
    sequence_output = transformer_output.last_hidden_state
    cls_token = sequence_output[:, 0, :]  # Use the [CLS] token from the last hidden state

    # Classification layer
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_token)
    output = keras.layers.Dense(units=4, activation="softmax")(logits)  # Assuming 4 classes for intent classification

    model = keras.Model(inputs=input_word_ids, outputs=output)
    return model


In [131]:

####### TOO USE #####
def build_model(transformer, max_len=100):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    # Get the CLS token (sequence_output)
    sequence_output = Lambda(lambda x: transformer(x)[0], output_shape=(max_len, 768))(input_word_ids)
    cls_token = sequence_output[:, 0, :]
    logits = keras.layers.Dense(units=768, activation="tanh")(cls_token)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=4, activation="softmax")(logits)

    model = Model(inputs=input_word_ids, outputs=logits)
    model.compile(optimizer=keras.optimizers.Adam(1e-5),loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")] )

    return model

In [132]:
######### TOOO USEEE #######

%%time
with strategy.scope():
    #transformer_model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')
    model = TFBertModel.from_pretrained('bert-base-cased')
    model = build_model(model, max_len=256)
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_word_ids (InputLayer)          │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_7 (Lambda)                    │ (None, 256, 768)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ get_item_3 (GetItem)                 │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 768)                 │         590,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 4)                   │           3,076 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 593,668 (2.26 MB)

 Trainable params: 593,668 (2.26 MB)

 Non-trainable params: 0 (0.00 B)

CPU times: user 6.13 s, sys: 658 ms, total: 6.79 s
Wall time: 4.37 s


# model train

In [133]:
EPOCHS = 20
n_steps = x_train.shape[0] // 16
train_history = model.fit(train_dataset, steps_per_epoch=n_steps,
    validation_data=valid_dataset, epochs=EPOCHS )

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:609: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


70/70 ━━━━━━━━━━━━━━━━━━━━ 739s 10s/step - acc: 0.2419 - loss: 1.5595 - val_acc: 0.2532 - val_loss: 1.3924
Epoch 2/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 717s 10s/step - acc: 0.3078 - loss: 1.4502 - val_acc: 0.3354 - val_loss: 1.3730
Epoch 3/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 703s 10s/step - acc: 0.2706 - loss: 1.4974 - val_acc: 0.2278 - val_loss: 1.3749
Epoch 4/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 700s 10s/step - acc: 0.2598 - loss: 1.5259 - val_acc: 0.2911 - val_loss: 1.3604
Epoch 5/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 700s 10s/step - acc: 0.2100 - loss: 1.5132 - val_acc: 0.3481 - val_loss: 1.3448
Epoch 6/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 698s 10s/step - acc: 0.2692 - loss: 1.4747 - val_acc: 0.3038 - val_loss: 1.3491
Epoch 7/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 700s 10s/step - acc: 0.2739 - loss: 1.4231 - val_acc: 0.3038 - val_loss: 1.3412
Epoch 8/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 699s 10s/step - acc: 0.2911 - loss: 1.4252 - val_acc: 0.3987 - val_loss: 1.3387
Epoch 9/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 746s 11s/step - acc: 0.2996 -

# model test

In [181]:
#x_test

array([[  101, 11336,  3702, ...,     0,     0,     0],
       [  101,  5619,  1155, ...,     0,     0,     0],
       [  101,   146,  1138, ...,     0,     0,     0],
       ...,
       [  101, 11513,   128, ...,     0,     0,     0],
       [  101, 19140,  2944, ...,     0,     0,     0],
       [  101,  3237,  1155, ...,     0,     0,     0]])

In [134]:
y_pred = model.predict(x_test).argmax(axis=-1)
y_pred

11/11 ━━━━━━━━━━━━━━━━━━━━ 345s 31s/step


array([2, 3, 3, 3, 0, 2, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 0,
       3, 3, 2, 0, 2, 3, 2, 0, 1, 3, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2,
       2, 2, 2, 2, 2, 3, 2, 2, 2, 0, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 3, 3,
       2, 2, 3, 2, 3, 2, 3, 2, 0, 3, 3, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
       2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 3,
       2, 3, 3, 3, 3, 3, 2, 3, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2,
       3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 0, 2, 2, 2, 1, 2, 3, 3,
       2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 3, 2, 2, 3, 3, 2, 3, 2, 2, 3,
       2, 3, 0, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 2, 3,
       2, 2, 2, 3, 2, 3, 2, 3, 3, 3, 0, 2, 2, 3, 2, 2, 1, 0, 3, 0, 2, 2,
       3, 2, 3, 2, 3, 2, 3, 3, 2, 3, 0, 2, 3, 2, 2, 3, 2, 0, 2, 2, 3, 2,
       2, 1, 0, 2, 2, 3, 2, 3, 2, 3, 2, 3, 3, 0, 3, 3, 2, 3, 2, 2, 2, 2,
       3, 2, 3, 2, 3, 2, 0, 2, 0, 3, 2, 2, 1, 2, 1, 2, 3, 3, 2, 3, 3, 3,
       3, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2,

In [135]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,test_y)


0.5217391304347826

In [136]:
from sklearn.metrics import classification_report
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.20      0.33        85
           1       1.00      0.14      0.25        63
           2       0.39      0.90      0.55        82
           3       0.63      0.74      0.68        92

    accuracy                           0.52       322
   macro avg       0.76      0.50      0.45       322
weighted avg       0.74      0.52      0.47       322



In [165]:
classes = ["add_task", "list_tasks", "complete_task", "delete_task"]

for text, label in zip(test.Text, y_pred):
  print("text:", text, "\nintent:", classes[label])
  print()

text: Remove the grocery task 
intent: complete_task

text: List all my current tasks 
intent: delete_task

text: I have completed task 4 
intent: delete_task

text: I want to see my tasks 
intent: delete_task

text: Create a task for calling John 
intent: add_task

text: Remove task about project deadline 
intent: complete_task

text: Mark task 3 as complete 
intent: delete_task

text: Erase task number 2 
intent: complete_task

text: Can I see the tasks I have? 
intent: delete_task

text: Erase task number 2 
intent: complete_task

text: Show all my tasks 
intent: delete_task

text: Give me the task list 
intent: complete_task

text: Remove task number 3 
intent: complete_task

text: Set a new task for paying bills 
intent: complete_task

text: Complete the task about groceries 
intent: complete_task

text: Delete the call John task 
intent: complete_task

text: Show all my tasks 
intent: delete_task

text: Include a task for car maintenance 
intent: complete_task

text: Can I see th

# function to find quotation marks

In [156]:
def quotationtext():
    # Taking input from the user
    user_input = input("Enter a string (with part in quotes): ")

    # Extracting the part within quotation marks
    start = user_input.find('"')  # Find the first quotation mark
    end = user_input.rfind('"')    # Find the last quotation mark

    # Check if both quotes are found
    if start != -1 and end != -1 and start < end:
        quoted_text = user_input[start + 1:end]  # Extract the text between quotes
        print("Extracted quoted text:", quoted_text)
    else:
        print("No valid quoted text found.")

quotationtext()

Enter a string (with part in quotes): ho 'k'
No valid quoted text found.


# main function

In [188]:
TASKS_FILE = "tasks.txt"

# Mapping of class labels to intents
LABELS = ["add_task", "list_tasks", "complete_task", "delete_task"]

def load_tasks():
    tasks = []
    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, "r") as file:
            tasks = [line.strip() for line in file.readlines()]
    return tasks

def save_tasks(tasks):
    with open(TASKS_FILE, "w") as file:
        file.write("\n".join(tasks) + "\n")

def add_task(description):
    tasks = load_tasks()
    tasks.append(description)
    save_tasks(tasks)
    return f"Task added: {description}"

def list_tasks():
    tasks = load_tasks()
    if not tasks:
        return "No tasks found."
    else:
        task_list = "Pending Tasks:\n"
        task_list += "\n".join([f"{i + 1}. {task}" for i, task in enumerate(tasks)])
        return task_list

def complete_task(task_number):
    tasks = load_tasks()
    if 0 < task_number <= len(tasks):
        completed_task = tasks.pop(task_number - 1)
        save_tasks(tasks)
        return f"Task completed: {completed_task}"
    else:
        return "Invalid task number."

def delete_task(task_number):
    tasks = load_tasks()
    if 0 < task_number <= len(tasks):
        deleted_task = tasks.pop(task_number - 1)
        save_tasks(tasks)
        return f"Task deleted: {deleted_task}"
    else:
        return "Invalid task number."

def classify_intent(text):
    #inputs = tokenizer(text, return_tensors="pt")
    print("input got:",text)
    input_encode = regular_encode(text,tokenizer,maxlen=MAX_LEN)
    #print("input encoding:",input_encode)
    pred = model.predict(x_test).argmax(axis=-1)
    #print(type(pred))
    #print('pred is ',pred)
    # Convert y_pred to a scalar if it's an array
    if isinstance(pred, (np.ndarray, list)):
        pred = pred[0]
    print('pred ',pred)
    intent = LABELS[pred]

    #outputs = model(**inputs)
    #probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    #predicted_class = torch.argmax(probs).item()
    #intent = LABELS[predicted_class]
    return intent

# Main function to handle CLI input
def main():
    print("Task Manager CLI! (Type 'exit' to quit)")
    while True:
        user_input = input("\n> ").strip()

        if user_input.lower() == "exit":
            print("exitiing ")
            break

        # Classify the user's intent
        intent = classify_intent(user_input)
        print("user intent classified as:",intent)

        if intent == "add_task":
            # Extract task description
            description = user_input.replace("add ", "").replace("task ", "").strip()
            print(add_task(description))

        elif intent == "list_tasks":
            print(list_tasks())

        elif intent == "complete_task":
            # Extract task number if available
            task_number = ''.join(filter(str.isdigit, user_input))
            if task_number.isdigit():
                print(complete_task(int(task_number)))
            else:
                print("Enter valid task number to be completed")

        elif intent == "delete_task":
            # Extract task number if available
            task_number = ''.join(filter(str.isdigit, user_input))
            if task_number.isdigit():
                print(delete_task(int(task_number)))
            else:
                print("Enter valid task number to delete")

        else:
            print("invalid command")

if __name__ == "__main__":
    main()


Welcome to the Task Manager CLI! Type 'exit' to quit.

> complete task 3


Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': Fal

input got: complete task 3
11/11 ━━━━━━━━━━━━━━━━━━━━ 338s 30s/step
pred  2
user intent classified as: complete_task
Invalid task number.

> exit
Goodbye!
